### Adafruit PCA9685 16-Channel Servo Tests

Checking out the servo comtrol board. You need to install the needed libraries using these commands in a terminal

```
sudo pip3 install adafruit-circuitpython-pca9685
sudo pip3 install adafruit-circuitpython-servokit
```



In [1]:
import time
import board
import busio
import adafruit_pca9685
from adafruit_servokit import ServoKit

i2c = busio.I2C(board.SCL, board.SDA)
pca = adafruit_pca9685.PCA9685(i2c)
kit = ServoKit(channels=16)

In [2]:
# Connected servos
servos=[0,1,2,3,4,5]

In [20]:
# Read all angles
for i in servos:
    print(f"Servo {i}: {kit.servo[i].angle}")

Servo 0: 89.81699877999186
Servo 1: 89.81699877999186
Servo 2: 84.54656364375762
Servo 3: 84.54656364375762
Servo 4: 89.81699877999186
Servo 5: 89.81699877999186


The same way we read the angles,we can move the servo to the needed angle (0 to 180)

In [27]:
kit.servo[0].angle=85

The issue is the servo always moves at full speed because the PWM signal just tells it the position to go to. This would make for a very jerky arm. One was around is to move in one degree intervals and that wait, move the next degree, wait and so on until we are at the final value.

Lets make a function. `channel` is the channel the servo is connected to, `angle` is the angle to move to and `speed` is the speed in degrees per second. E.g. moving from 0 to 90 degrees at 45 degrees per second would take two seconds. 

In [79]:
def moveTo(channel,angle,speed):
    
    start  = kit.servo[channel].angle    
    points = abs(round(angle-start))
    if (points>0):
        delta  = (angle-start)/points
        for p in range(points):
            start=start+delta
            kit.servo[channel].angle = start
            time.sleep(1.0/speed)
    
    kit.servo[channel].angle = angle
    #print (f"Final angle: { kit.servo[channel].angle}")

In [60]:
moveTo(2,108.1,45)

Final angle: 107.9707198047987


### Arm Controller

In [89]:
import ipywidgets.widgets as widgets

In [166]:
class joint:
    # init method or constructor
    def __init__(self, name, servo_id, home=90, max_value=180, min_value=0, speed=30):
         
        # Instance Variable
        self.name = name
        self.servo_id = servo_id
        self.max_value = max_value
        self.min_value = min_value
        self.speed = speed
        self.home = home

joints = [joint("rotation", 5),
          joint("lower arm", 2),
          joint("upper arm", 4),
          joint("wrist rotation", 1),
          joint("wrist tilt", 0),
          joint("gripper", 3)]

In [169]:
def arm_home():
    for joint in joints:
        moveTo(joint.servo_id,joint.home,joint.speed)

In [215]:
# Array to save points (in joints order)
stored_points=[]

Callback from UI.

In [167]:
def handle_slider_change(change):
    for joint in joints:
        if(joint.name == change.owner.description):
            moveTo(joint.servo_id,change.new,joint.speed)

In [177]:
def home_button_pressed(button):
    arm_home()
    update_sliders()

In [223]:
def save_button_pressed(button):
    point = [kit.servo[j.servo_id].angle for j in joints]
    stored_points.append(point)
    message.value = f"Stored points: {len(stored_points)}"

In [234]:
def reset_button_pressed(button):
    stored_points.clear()
    message.value = f"Stored points: {len(stored_points)}"

In [284]:
def play_button_pressed(button):
    for angles in stored_points:
        for i,joint in enumerate(joints):
            if(abs(kit.servo[joint.servo_id].angle - angles[i]) > 0.1):
                moveTo(joint.servo_id,angles[i],joint.speed)
                message.value = f"Moving to point {i}"
                time.sleep(0.5)
                
    message.value = f"Done"

In [285]:
def update_sliders():
    for i, joint in enumerate(joints):
        sliders[i].value = kit.servo[joint.servo_id].angle

In [286]:
sliders=[]
for joint in joints:
    sliders.append(widgets.FloatSlider(value=kit.servo[joint.servo_id].angle, 
                                       min=joint.min_value, max=joint.max_value, step=1, description=joint.name,
                                       disabled=False, continuous_update=False, orientation='horizontal',
                                       readout=True, readout_format='.1f',layout=widgets.Layout(width='90%', height='50px')))
    sliders[-1].observe(handle_slider_change, names='value')

In [290]:
home_button = widgets.Button(description='Home', button_style='danger', layout=widgets.Layout(width='80%', height='40px'))
home_button.on_click(home_button_pressed)
save_button = widgets.Button(description='Save', button_style='success', layout=widgets.Layout(width='80%', height='40px'))
save_button.on_click(save_button_pressed)
reset_button = widgets.Button(description='Reset', button_style='warning', layout=widgets.Layout(width='80%', height='40px'))
reset_button.on_click(reset_button_pressed)
play_button = widgets.Button(description='Play', button_style='danger', layout=widgets.Layout(width='80%', height='40px'))
play_button.on_click(play_button_pressed)
message = widgets.Label(value="-", layout=widgets.Layout(align_self='center'))
message.value = f"Stored points: {len(stored_points)}"

In [291]:
panel_sliders = widgets.VBox(sliders, layout=widgets.Layout(width='80%'))
panel_buttons = widgets.VBox([home_button, save_button, reset_button, play_button, message],layout=widgets.Layout(width='15%'))
display(widgets.HBox([panel_sliders,panel_buttons]))